# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports, Installs**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b model/prompting origin/model/prompting

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet

/content
Cloning into 'project'...
remote: Enumerating objects: 981, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 981 (delta 71), reused 55 (delta 54), pack-reused 824 (from 1)
Receiving objects: 100% (981/981), 1.73 MiB | 7.61 MiB/s, done.
Resolving deltas: 100% (525/525), done.
/content/project
Branch 'model/prompting' set up to track remote branch 'model/prompting' from 'origin'.
Switched to a new branch 'model/prompting'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.6/433.6 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

**--Default imports**

In [2]:
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key


**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

**--Lauren's bit that can run the feedback generator here to test**
- ended up modularizing code to prepare for gradio/UI use.
  - had all files include ability to user uploaded files, used local testing files as default to be able to test
  - created feedback_engine file to be ablet to create runnable program in notebook to test
  - TODO:: create runable feedback programe for Gradio/UI when that time comes

In [10]:
from tropos import test_feedback_console

test_feedback_console(
    requirements_path="/content/project/data/raw/Requirements.docx",
    example_dir="/content/project/data/raw",
    target_dir="/content/project/data/unmarked_raw",
    output_dir="/content/project/data/generated_output",
    verbose=True
)


No tables in document.
Document contains no comments
No tables in document.
Document contains no comments
No tables in document.
Document contains no comments
No tables in document.
Document contains no comments
--- FewShot Feedback for Uncommented_Student_1 ---
📝 Instructor Feedback

🗨️  "Unfortunately, the year 2019 is known as the year of quarantine." 👉 – While
2019 marked the emergence of COVID-19, widespread quarantines were more
characteristic of 2020. Consider rephrasing for greater accuracy.

🗨️  "A year later, the public questioned governmental enforcement regarding the
mask mandate." 👉 – This sentence could benefit from more context or examples of
how public questioning manifested.

🗨️  "Enforcement is the key difference, this will be explained later in detail."
👉 – This sentence is a run-on. Consider splitting it into two distinct sentences
for clarity.

🗨️  "The noncompliant side questions the true effectiveness of the mask
mandate." 👉 – It would be helpful to include speci